<a href="https://colab.research.google.com/github/dawidmakowski-git/tensorflow_tutorials/blob/master/Introduction_to_the_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 1.8MB/s 


# Download and prepare the dataset

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# Define the model

In [5]:
# When you build a model for hypertuning, 
# you also define the hyperparameter search space
# in addition to the model architecture. 
# The model you set up for hypertuning is called a hypermodel.

def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

# **Instantiate the tuner and perform hypertuning**

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')  

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.001.



In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4790 - accuracy: 0.8277 - val_loss: 0.4634 - val_accuracy: 0.8317
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3609 - accuracy: 0.8684 - val_loss: 0.3834 - val_accuracy: 0.8626
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3229 - accuracy: 0.8795 - val_loss: 0.3447 - val_accuracy: 0.8763
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2995 - accuracy: 0.8869 - val_loss: 0.3463 - val_accuracy: 0.8761
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2819 - accuracy: 0.8948 - val_loss: 0.3497 - val_accuracy: 0.8786
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2685 - accuracy: 0.8997 - val_loss: 0.3387 - val_accuracy: 0.8754
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2524 - accuracy: 0.9035 - val_loss: 0.3312 - val_accuracy: